**직렬화란 모델을 저장 가능한 형식으로 변환하는 과정**

모델 재사용 (재훈련 없이)


모델 배포 및 공유 용이


계산 리소스 절약

In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

**직렬화가 가능한지 체크**

In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿을 사용하여 질문을 생성합니다.
prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")
# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 체인을 생성합니다.
chain = prompt | llm

# 직렬화가 가능한지 체크합니다.
chain.is_lc_serializable()

True

**직렬화 방법**

dumps: 객체를 JSON 문자열로 직렬화 


dumpd: 객체를 딕셔너리로 직렬화

In [5]:
from langchain_core.load import dumpd, dumps

dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate',
   'graph': {'nodes': [{'id': 0, 'type': 'schema', 'data': 'PromptInput'},
     {'id': 1,
      'type': 'runnable',
      'data': {'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
       'name': 'PromptTemplate'}},
     {'id': 2, 'type': 'schema', 'data': 'PromptTemplateOutput'}],
    'edges': [{'source': 0, 'target': 1}, {'source': 1, 'target': 2}]}},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'openai_p

In [6]:
# dumps 함수를 사용하여 직렬화된 체인을 확인합니다.
dumps_chain = dumps(chain)
dumps_chain

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate", "graph": {"nodes": [{"id": 0, "type": "schema", "data": "PromptInput"}, {"id": 1, "type": "runnable", "data": {"id": ["langchain", "prompts", "prompt", "PromptTemplate"], "name": "PromptTemplate"}}, {"id": 2, "type": "schema", "data": "PromptTemplateOutput"}], "edges": [{"source": 0, "target": 1}, {"source": 1, "target": 2}]}}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-3.5-turbo", "temperature": 0.0, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}, "openai_proxy": "", "max_retries": 2

In [8]:
print("dumpd :", type(dumpd_chain))
print("dumps :", type(dumps_chain))

dumpd : <class 'dict'>
dumps : <class 'str'>


**이제 pickle을 사용해서 저장해줍니다.**

In [9]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)


# json 으로 저장도 가능
# import json

# with open("fruit_chain.json", "w") as fp:
#     json.dump(dumpd_chain, fp)

load 해서 사용

In [10]:
from langchain_core.load import load

with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)
# 체인을 로드합니다.
chain_from_file = load(loaded_chain)

# 체인을 실행합니다.
print(chain_from_file.invoke({"fruit": "사과"}))

c:\Users\kyr76\OneDrive\바탕 화면\langchain_study\llm\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.' response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-92170362-cc45-4636-9c87-b8543712ab1a-0' usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75}
